# Question 1: Design and describe an architecture for DCGAN. 

Train DCGAN using
MNIST training examples (60K) and perform evaluation experiments.
Try some different architectures, hyper-parameters, and, if necessary, the aspects of virtual batch
normalization, balancing G and D.
Please discuss, with results, what challenge and how they are specifically addressing, including
the quality of generated images and, also, the mode collapse. 

## Imports of Modules

In [1]:
import sys
import sys, json, os, glob, time, cv2, random
import keras
from keras import backend as K
from keras.utils import plot_model, np_utils

import tensorflow as tf
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from load_data import *
from dcgan import *
from post_process import *
import json

Using TensorFlow backend.


## Fix seeds

In [2]:
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

## Load data

In [3]:
X_train, Y_train, X_test, Y_test = load_data()

## Experiments
We define the hyperparameters that we are going to change:

1. Filter capacity can be 0.25, 1, 1.75 than the original
2. The ratio of discriminator to generator 1:1, 2:1, 1:2
3. The batch size being 64, 128, 256

Then after finding the best model we are going to try and use label smoothing and virtual batch normalization to see what is going to be the outcome. 

In [4]:
filters =[1,2,3]
depth = [1,2,3]
batch_size = [32, 64, 128]
epochs = 25
train = False

In [7]:
models = {}
for f in filters:
    for d in depth:
        for b in batch_size:
            # We run out of memory for these models 
            if (d ==3 and f ==3) or (b == 128 and f == 3 and d == 3) or (f==3 and (b==64 or b == 128) and d ==2):
                continue
            name = "DCGAN_{}_{}_{}".format(f,d,b)
            history = None
            model = DCGAN(name, d, f)
            if train:
                history = model.train(X_train,
                                      epochs=epochs, batch_size = b,
                                      sample_interval= 5,
                                      save = True)
                models[name] = history
            else:
                models[name] = model.load_model("models/{}".format(name))
            # Empty the memory
            del model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_113 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_191 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_141 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_114 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_192 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_142 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_29 (Reshape)         (None, 7, 7, 128)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_121 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_203 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_151 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_122 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_204 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_152 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_31 (Reshape)         (None, 7, 7, 128)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_129 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_216 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_161 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_130 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_217 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_162 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_33 (Reshape)         (None, 7, 7, 128)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_137 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_230 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_171 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_138 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_231 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_172 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_35 (Reshape)         (None, 7, 7, 128)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_145 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_246 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_181 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_146 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_247 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_182 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_37 (Reshape)         (None, 7, 7, 128)         0         
__________

...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_161 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_272 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_201 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_162 (Dense)            (None, 12544)             12857600  
_________________________________________________________________
batch_normalization_273 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_202 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_41 (Reshape)         (None, 7, 7, 256)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_169 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_286 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_211 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_170 (Dense)            (None, 12544)             12857600  
_________________________________________________________________
batch_normalization_287 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_212 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_43 (Reshape)         (None, 7, 7, 256)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_177 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_301 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_221 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_178 (Dense)            (None, 12544)             12857600  
_________________________________________________________________
batch_normalization_302 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_222 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_45 (Reshape)         (None, 7, 7, 256)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_185 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_317 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_231 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_186 (Dense)            (None, 18816)             19286400  
_________________________________________________________________
batch_normalization_318 (Bat (None, 18816)             75264     
_________________________________________________________________
activation_232 (Activation)  (None, 18816)             0         
_________________________________________________________________
reshape_47 (Reshape)         (None, 7, 7, 384)         0         
__________

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_193 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_329 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_241 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_194 (Dense)            (None, 18816)             19286400  
_________________________________________________________________
batch_normalization_330 (Bat (None, 18816)             75264     
_________________________________________________________________
activation_242 (Activation)  (None, 18816)             0         
_________________________________________________________________
reshape_49 (Reshape)         (None, 7, 7, 384)         0         
__________

## Plot losses

All the data will be located in the Figures directory in the main folder. 

In [8]:
discriminator_losses = []
discriminator_accuracies = []
generator_losses = []
names = []
for name, data in models.items():
    names.append(name)
    discriminator_losses.append(data['discriminator_loss'])
    discriminator_accuracies.append(data['discriminator_accuracy'])
    generator_losses.append(data['generator_loss'])
    
plot_graphs(discriminator_losses, discriminator_accuracies, generator_losses, names, "../../Figures/CW2/Q1/normal/")

## Virtual Batch Normalisation
Now we are going to apply virtual batch normalisation to the most successful model

In [9]:
f = 1
b = 32
d = 1
epochs = 20
name = "DCGAN_{}_{}_{}_vb".format(f,d,b)
history = None
model = DCGAN(name, d, f)
model.virtual_batch_norm = True
train = False
if train:
    history = model.train(X_train,
                          epochs=epochs, batch_size = b,
                          sample_interval= 5,
                          save = True)
else:
    history = model.load_model("models/{}".format(name))
# Empty the memory
del model

discriminator_losses = [list(history['discriminator_loss'])]
discriminator_accuracies = [list(history['discriminator_accuracy'])]
generator_losses = [list(history['generator_loss'])]
names = [name]

plot_graphs(discriminator_losses, discriminator_accuracies, generator_losses, names,"../../Figures/CW2/Q1/vb/")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_201 (Dense)            (None, 1024)              103424    
_________________________________________________________________
batch_normalization_342 (Bat (None, 1024)              4096      
_________________________________________________________________
activation_251 (Activation)  (None, 1024)              0         
_________________________________________________________________
dense_202 (Dense)            (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_343 (Bat (None, 6272)              25088     
_________________________________________________________________
activation_252 (Activation)  (None, 6272)              0         
_________________________________________________________________
reshape_51 (Reshape)         (None, 7, 7, 128)         0         
__________